In [56]:
from wit import Wit
import pandas as pd

In [86]:
def loadData():
    return pd.read_csv("/Users/karchugh/Projects/Otto/src/static/datasets/tweets.csv", encoding="latin-1")

In [129]:
FEATURE_COLUMN = 1
DATA_COUNT = 5

In [130]:
data = loadData()

In [131]:
# Selects column to analyze
data = data.iloc[:, FEATURE_COLUMN]
# Keeps specified number of rows
data = data.sample(n=DATA_COUNT, random_state=42)

In [132]:
T = "GUNUX2F3ZIXVVLK3DCRY6NXH2OLGOG6I"
client = Wit(T)

In [147]:
def traitLabelForSample(traits):
    traitArr = []
    for traitName in traits:
        matches = traits[traitName]
        firstMatch = matches[0]
        value = firstMatch['value']
        traitArr.append(traitName + ": " + value)
    traitLabel = ', '.join(traitArr)
    return traitLabel

In [148]:
def entityLabelForSample(entities):
    entityArr = []
    for entityName in entities:
        matches = entities[entityName]
        firstMatch = matches[0]
        value = firstMatch['body']
        entityArr.append(entityName + ": " + value)
    entityLabel = ', '.join(entityArr)
    return entityLabel

In [155]:
texts = []
entityLabels = []
traitLabels = []

for i, row in data.iteritems():
    text = str(row)
    resp = client.message(text)

    entities = resp.get('entities')
    traits = resp.get('traits')
    entityLabel = entityLabelForSample(entities)
    traitLabel = traitLabelForSample(traits)

    entityLabels.append(entityLabel)
    traitLabels.append(traitLabel)
    texts.append(text)

In [153]:
entityLabels

['wit$contact:contact: @',
 'wit$contact:contact: @a02toyota',
 '',
 'wit$contact:contact: SomersetMarcy',
 'wit$contact:contact: assistant']

In [154]:
traitLabels

['',
 'wit$thanks: true, wit$sentiment: positive',
 'wit$sentiment: negative',
 'wit$sentiment: positive',
 'wit$sentiment: neutral']

In [168]:
labeled = pd.DataFrame({'a':texts,'b':entityLabels})

In [23]:
r

{'text': 'cya guys tomorrow and thank you for everything',
 'intents': [],
 'entities': {'wit$datetime:datetime': [{'id': '248762126419129',
    'name': 'wit$datetime',
    'role': 'datetime',
    'start': 9,
    'end': 17,
    'body': 'tomorrow',
    'confidence': 0.9672,
    'entities': [],
    'type': 'value',
    'grain': 'day',
    'value': '2020-06-19T00:00:00.000-07:00',
    'values': [{'type': 'value',
      'grain': 'day',
      'value': '2020-06-19T00:00:00.000-07:00'}]}]},
 'traits': {'wit$thanks': [{'id': '5900cc2d-867c-4bf0-beb6-ef6e6bf45ca5',
    'value': 'true',
    'confidence': 0.5571}],
  'wit$sentiment': [{'id': '5ac2b50a-44e4-466e-9d49-bad6bd40092c',
    'value': 'positive',
    'confidence': 0.9507}]}}